<a href="https://colab.research.google.com/github/epj99/IBM-Data-Analyst-Capstone-2026/blob/main/Engineering_Gantt_Project_Audit_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Project Audit Tool with Python

# Engineering Project Audit Tool: Predictive Risk Detection
**Author:** Esteban Piano Jaramillo  
**Role:** Senior Project Manager | Junior Data Analyst  

### 1. The Executive Problem
In heavy industrial projects, such as the installation of a food processing line, schedule
delays are rarely caused by a lack of effort. They are caused by **latent logic failures** and **physical constraint violations** that are invisible in massive Gantt charts.

### 2. The Solution
This tool leverages Python and Pandas to perform an automated "Health Check" on project
schedules. It doesn't just look at dates; it applies **Engineering Domain Expertise** to identify risks such as:
* **Logical Loops:** Circular dependencies that break critical path calculations.
* **Site Access Violations:** Scheduling technical work during physical constraints
  (e.g., concrete curing).
* **Vague Tasking:** Identifying "Black Hole" tasks that lack the granularity needed
  for effective control.

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import plotly.express as px
import re

# 1. Load the file
file_path = '/content/Excel Project Schedule Template & Python.xlsx'
df_raw = pd.read_excel(file_path)

def prepare_gantt_data(df):
    df = df.copy()
    # Standardize column names
    df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]
    # Convert dates
    df['start_date'] = pd.to_datetime(df['start_date'])
    df['finish_date'] = pd.to_datetime(df['finish_date'])
    # Clean progress
    if df['%_complete'].dtype == object:
        df['progress_decimal'] = df['%_complete'].str.rstrip('%').astype('float') / 100.0
    else:
        df['progress_decimal'] = df['%_complete'] / 100.0
    return df

df_clean = prepare_gantt_data(df_raw)
print("Data successfully loaded and cleaned.")
df_clean.head()

Data successfully loaded and cleaned.


,task_id,task_name,start_date,finish_date,duration_(days),predecessors,%_complete,resource,progress_decimal
0,100,Project Kick-off,2026-03-01,2026-03-01,0,NaN,1.0,PM Team,0.010
1,110,Soil Study & Site Prep,2026-03-02,2026-03-12,10,100,1.0,Civil Sub,0.010
2,120,Foundation Pouring,2026-03-13,2026-03-20,7,110,0.5,Civil Sub,0.005
3,130,Concrete Curing (Lag),2026-03-21,2026-03-28,7,120FS+7,0.0,NaN,0.000
4,200,Steel Procurement,2026-03-02,2026-03-30,28,100,0.2,Procurement,0.002


3. Domain-Specific Audit: The "Curing Window" Conflict
In industrial construction, site accessibility is a physical constraint that is often overlooked.

The Engineering Problem: Concrete pouring and curing create a "No-Go" zone. Specifically, electrical wall chasing (Task 330) and MDB installation (Task 340) cannot be performed while the concrete is wet.

The Risk: Scheduling these during the curing window leads to Standby Labor Costs and safety violations.

4: Audit Execution (Code)

In [3]:
def run_full_audit(df):
    curing_start = pd.to_datetime('2026-03-13')
    curing_end = pd.to_datetime('2026-03-28')
    site_dependent = [330, 340]

    print("--- 🚩 PROJECT AUDIT REPORT 🚩 ---")

    # Check for Site Access Violations
    for _, row in df.iterrows():
        if row['task_id'] in site_dependent:
            if (row['start_date'] <= curing_end) and (row['finish_date'] >= curing_start):
                print(f"❌ ERROR [Site Access]: Task {row['task_id']} ({row['task_name']}) "
                      f"overlaps with Concrete Curing (Ends {curing_end.date()}).")

    # Check for "Black Hole" Tasks (> 30 days)
    long_tasks = df[df['duration_(days)'] > 30]
    for _, row in long_tasks.iterrows():
        print(f"⚠️ WARNING [Vague Duration]: Task {row['task_id']} lasts {row['duration_(days)']} days.")

run_full_audit(df_clean)

--- 🚩 PROJECT AUDIT REPORT 🚩 ---
❌ ERROR [Site Access]: Task 330 (Wall Chasing & Conduit Path Preparation) overlaps with Concrete Curing (Ends 2026-03-28).
❌ ERROR [Site Access]: Task 340 (Installation and termination of Main Distribution Board (MDB).) overlaps with Concrete Curing (Ends 2026-03-28).
⚠️ WARNING [Vague Duration]: Task 520 lasts 70 days.


### 4. Domain-Specific Audit: The "Curing Window" Conflict
In industrial construction, site accessibility is a physical constraint that is often
overlooked in digital schedules.

**The Engineering Problem:** Concrete pouring (Task 120) and curing (Task 130) create a "No-Go" zone for other trades.
Specifically, electrical wall chasing (Task 330) and MDB installation (Task 340) cannot
be performed while the concrete is wet or curing due to safety, moisture, and structural
integrity protocols.

**The Risk:** If these activities overlap, the project faces:
1. **Standby Labor Costs:** Subcontractors arrive but cannot work.
2. **Quality Risks:** Dust from wall chasing contaminating fresh concrete.
3. **Safety Violations:** High-voltage work in high-moisture environments.

The following code block executes a **Site Access Audit** to flag these specific
overlaps using my memory-based project constraints.

In [4]:
def run_full_audit(df):
    """
    Executes the risk detection engine.
    """
    # Define the Curing Window (From your project memory)
    curing_start = pd.to_datetime('2026-03-13')
    curing_end = pd.to_datetime('2026-03-28')

    # Define Site-Dependent Tasks
    site_dependent = [330, 340]

    print("--- 🚩 PROJECT AUDIT REPORT 🚩 ---")

    # 1. Check for Site Access Violations
    for _, row in df.iterrows():
        if row['task_id'] in site_dependent:
            if (row['start_date'] <= curing_end) and (row['finish_date'] >= curing_start):
                print(f"❌ ERROR [Site Access]: Task {row['task_id']} ({row['task_name']}) "
                      f"overlaps with Concrete Curing (Ends {curing_end.date()}).")

    # 2. Check for the Logic Loop (330 <-> 340)
    # We will expand this with a formal loop-detector in the next phase
    print("\n--- 🔍 LOGIC VALIDATION ---")
    print("Checking for recursive dependencies...")
    # Add your specific loops here as we build the parser

    # 3. Check for "Black Hole" Tasks (> 30 days)
    long_tasks = df[df['duration_(days)'] > 30]
    for _, row in long_tasks.iterrows():
        print(f"⚠️ WARNING [Vague Duration]: Task {row['task_id']} lasts {row['duration_(days)']} days. "
              "Split into sub-tasks for better control.")

# Execute the report
run_full_audit(df_clean)

--- 🚩 PROJECT AUDIT REPORT 🚩 ---
❌ ERROR [Site Access]: Task 330 (Wall Chasing & Conduit Path Preparation) overlaps with Concrete Curing (Ends 2026-03-28).
❌ ERROR [Site Access]: Task 340 (Installation and termination of Main Distribution Board (MDB).) overlaps with Concrete Curing (Ends 2026-03-28).

--- 🔍 LOGIC VALIDATION ---
Checking for recursive dependencies...
⚠️ WARNING [Vague Duration]: Task 520 lasts 70 days. Split into sub-tasks for better control.


### 5. Advanced Audit: Recursive Dependency Detection (Logic Loops)
A project schedule is essentially a **Directed Acyclic Graph (DAG)**. For critical path
calculations to work, the logic must flow in one direction.

**The Engineering Problem:** "Logic Loops" (Circular Dependencies) occur when Task A
depends on B, and B depends on A. This creates a mathematical paradox where the software
cannot calculate a start date.

**The Data Solution:** Using **Graph Theory** and the `networkx` library, I represent
the schedule as a system of Nodes (Tasks) and Edges (Dependencies). By applying a
Cycle Detection algorithm, the tool automatically flags exactly where the logic "folds"
on itself, ensuring the schedule is mathematically sound.

In [6]:
import networkx as nx

def detect_logic_loops(df):
    """
    Uses Graph Theory to find circular dependencies in the schedule.
    Corrected for NetworkX syntax.
    """
    G = nx.DiGraph()

    # 1. Add all tasks as nodes
    for _, row in df.iterrows():
        G.add_node(row['task_id'])

    # 2. Add edges (dependencies)
    for _, row in df.iterrows():
        preds = str(row['predecessors']).split(',')
        for p in preds:
            # Extract just the numbers (e.g., '120FS+7' -> '120')
            p_id = "".join(filter(str.isdigit, p.strip()))
            if p_id:
                # Add edge: Predecessor -> Current Task
                G.add_edge(int(p_id), row['task_id'])

    # 3. Detect Cycles
    try:
        # find_cycle returns the first cycle it encounters
        cycle = nx.find_cycle(G, orientation="original")
        if cycle:
            print("❌ CRITICAL ERROR: Logic Loop Detected!")
            for start, end, _ in cycle:
                print(f"   Loop Link: Task {start} ---> Task {end}")
    except nx.NetworkXNoCycle:
        print("✅ Logic Check: No circular dependencies found.")

# Run the detector
detect_logic_loops(df_clean)

❌ CRITICAL ERROR: Logic Loop Detected!
   Loop Link: Task 330 ---> Task 340
   Loop Link: Task 340 ---> Task 330


### 6. Interactive Risk Visualization: The PM Decision Dashboard
Data is only useful if it leads to action. In this final stage, I visualize the project
timeline and overlay the identified risks using **Plotly**.

**Key Features:**
* **Color-Coded Status:** Tasks are automatically flagged (Red for Logic Loops/Access Violations, Yellow for Vague Durations).
* **Hover Interaction:** Users can see specific audit notes by hovering over task bars.
* **Timeline Clarity:** The dashboard provides a clear view of how the "Concrete Curing"
  window impacts the planned electrical activities.

In [8]:
import plotly.express as px

def create_risk_dashboard(df):
    """
    Creates an interactive Plotly Gantt chart highlighting project risks.
    Corrected for Plotly Express 'x_start' and 'x_end' arguments.
    """
    # 1. Prepare a 'Risk Level' column for coloring
    df['risk_category'] = 'Healthy'

    # Flag the traps we built (Ensure IDs match your file)
    df.loc[df['task_id'].isin([330, 340]), 'risk_category'] = 'Critical: Logic/Site Conflict'

    # Use the flexible duration column check we used in the audit
    dur_col = [col for col in df.columns if 'duration' in col][0]
    df.loc[df[dur_col] > 30, 'risk_category'] = 'Warning: Vague Duration'

    # 2. Build the chart
    fig = px.timeline(
        df,
        x_start="start_date",
        x_end="finish_date",
        y="task_name",
        color="risk_category",
        color_discrete_map={
            'Healthy': '#2ecc71', # Green
            'Critical: Logic/Site Conflict': '#e74c3c', # Red
            'Warning: Vague Duration': '#f1c40f' # Yellow
        },
        hover_data=['task_id', dur_col, 'predecessors'],
        title="<b>Interactive Engineering Project Audit Dashboard</b>",
        labels={'task_name': 'Project Activity'}
    )

    # 3. Add the 'Concrete Curing' shaded area (March 13 to March 28)
    fig.add_vrect(
        x0="2026-03-13", x1="2026-03-28",
        fillcolor="gray", opacity=0.2,
        layer="below", line_width=0,
        annotation_text="Concrete Curing Window",
        annotation_position="top left"
    )

    fig.update_yaxes(autorange="reversed") # Standard Gantt view (top-to-bottom)
    fig.update_layout(template="plotly_white", height=600)

    fig.show()

# Run the final dashboard
create_risk_dashboard(df_clean)

### 7. Executive Summary & Audit Findings
The automated audit of the **Food Processing Line Installation** project has identified
three critical risk categories that require immediate corrective action:

1. **Site Access Violations (Financial Risk):** Tasks 330 and 340 (Electrical) are scheduled during the Concrete Curing window
(March 13 - 28). This would result in an estimated **€15,000+ in standby labor costs** as the site is physically inaccessible during this period.

2. **Mathematical Paradox (Logic Loops):** A circular dependency was detected between Tasks 330 and 340. This prevents the
scheduling engine from calculating a valid Critical Path (CPM), making the total
project duration unreliable.

3. **Control Gaps (Vague Durations):** Task 520 (70 days) is a "Black Hole." In senior project management, any task over
20 days must be decomposed to ensure accurate progress tracking and earned value
management.

**Recommendation:** Adjust the electrical start dates to post-March 28, resolve the recursive logic loop
in the WBS, and decompose Task 520 into identifiable milestones.